# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming


## Install required packages 

In [1]:
!pip install pandas nltk numpy matplotlib


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import regex as re
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zaccu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zaccu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load dataset

In [3]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [4]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [5]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [6]:
dev_set['premise_tokens'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_tokens'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_tokens'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_tokens'] = train_set['hypothesis'].apply(clean_text)

In [7]:
dev_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]"
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]"
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei..."
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l..."
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]"


In [8]:
train_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1,"[yeah, know, cut, california, half, something]","[yeah, sure, make, fit, maybe, could, cut, cal..."
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1,"[actual, name, used]","[sake, privacy, actual, name, used]"
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1,"[film, directed, randall, wallace]","[film, directed, randall, wallace, star, mel, ..."
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1,"[know, sign, anse, studied, unkempt, clean, re...","[anse, looked, cracked, mirror]"
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1,"[light, candle, cheek, looked, even, hollow, t...","[drew, regarded, best, friend, noted, light, l..."


Dataset analysis

In [9]:
# Labels = dev_set['label'].unique()
# Labels

# def get_word_frequency(data):
#     word_freq = {}
#     for row in data:
#         for word in row:
#             if word in word_freq:
#                 word_freq[word] += 1
#             else:
#                 word_freq[word] = 1
#     return word_freq

# word_freq = get_word_frequency(train_set['premise_tokens'] + train_set['hypothesis'])

# # nltk FreqDist
# from nltk import FreqDist

# fdist = FreqDist(word_freq)
# fdist

# embeddings/ vectorization

In [10]:
glove = "./glove_embeddings/glove.6B.200d.txt"
def load_glove(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)  # <-- Convert to float32
            embeddings_dict[word] = vector
    return embeddings_dict

embedding_dim = 200
loaded_glove = load_glove(glove)



In [ ]:
def sentence_embedding(tokens, embeddings_dict, embedding_dim):
    
    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]

    if not valid_embeddings:
        # Return zero-vector if no embeddings found
        return np.zeros(embedding_dim)
    sentence_emb = np.mean(valid_embeddings, axis=0)
    return sentence_emb

def pairwise_embedding(premise_tokens, hypothesis_tokens, embeddings_dict,embedding_dim):
    premise_emb = sentence_embedding(premise_tokens, embeddings_dict,embedding_dim)
    hypothesis_emb = sentence_embedding(hypothesis_tokens, embeddings_dict,embedding_dim)
    # Concatenate multiple useful features
    combined_emb = np.concatenate([
        premise_emb,
        hypothesis_emb,
        np.abs(premise_emb - hypothesis_emb), # capture difference
        premise_emb * hypothesis_emb           # capture interactions
    ]).astype(np.float32) 
    
    return combined_emb 

    

In [13]:
from tqdm import tqdm
tqdm.pandas()

# Ensure correct embeddings are used for processing
train_set['combined_embedding'] = train_set.progress_apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], loaded_glove, embedding_dim=300), 
    axis=1
)

dev_set['combined_embedding'] = dev_set.progress_apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], loaded_glove, embedding_dim=300), 
    axis=1
)


  0%|          | 7/24432 [00:00<07:01, 57.99it/s]

['yeah', 'know', 'cut', 'california', 'half', 'something']
[array([ 5.8852e-01,  2.3437e-01,  1.5102e-02,  1.5420e-01, -6.6995e-01,
        2.2334e-01, -5.9206e-01,  2.5223e-01,  2.1026e-01,  5.4238e-01,
       -4.4113e-01,  4.3027e-02,  4.0226e-01,  2.6381e-01, -5.5256e-01,
        4.6819e-01, -9.2069e-01,  3.4615e-01,  4.1744e-01,  2.9842e-02,
        5.6711e-01,  1.4103e-01, -4.9243e-02, -4.0423e-01, -1.2173e-01,
       -8.8096e-01,  1.5899e-01,  2.4584e-01, -4.5463e-01, -6.1556e-01,
       -5.1147e-01, -3.2836e-01, -3.1620e-01, -2.1710e-01, -2.9038e-01,
       -1.1232e-01, -8.1165e-03, -3.8394e-01, -1.9524e-01,  5.4950e-01,
       -6.0194e-01, -2.2716e-01, -5.1569e-01,  2.9571e-01,  1.0878e-01,
       -2.9637e-01,  6.4856e-01, -1.0606e-01, -1.9638e-01,  9.0679e-02,
        7.3177e-01,  1.1545e-01, -1.5148e-01,  6.2442e-02, -2.9770e-01,
       -9.1699e-02, -3.4336e-02,  1.9295e-01, -6.5724e-01, -3.6813e-01,
        2.3374e-01, -3.1870e-01, -9.0369e-02,  9.8264e-02, -1.6097e-01,
    

  0%|          | 22/24432 [00:00<06:26, 63.10it/s]

[array([-0.13128  ,  0.47094  , -0.41581  , -0.57719  ,  0.4084   ,
       -0.074808 , -0.5941   ,  0.27978  ,  0.35669  ,  0.14344  ,
       -0.12186  ,  0.41693  ,  0.18581  ,  0.016802 ,  0.33867  ,
       -0.46874  , -0.10281  ,  0.57245  , -0.94986  ,  0.23983  ,
        0.26144  ,  2.4242   , -0.38137  ,  0.035984 ,  0.19365  ,
        0.33187  ,  0.18094  ,  0.11304  ,  0.045898 , -0.023212 ,
       -0.81736  , -0.18159  ,  0.57427  ,  0.50874  ,  0.41294  ,
        0.62009  , -0.27301  , -0.25676  , -0.32071  , -0.24978  ,
       -0.11106  , -0.56452  ,  0.0076912,  0.87061  ,  0.17609  ,
        0.24631  , -0.10875  , -0.50961  ,  0.10031  , -0.045482 ,
        0.0061603, -0.24677  ,  0.18008  , -0.16468  , -0.024297 ,
       -0.083325 , -0.27346  ,  0.12345  ,  0.04623  , -0.037474 ,
        0.64581  ,  0.033174 , -0.061225 , -0.23671  , -0.22095  ,
       -0.49998  ,  0.078095 ,  0.22458  ,  0.13711  ,  0.21514  ,
       -0.13537  ,  0.56045  , -0.07697  ,  0.20533  ,  0.094

  0%|          | 48/24432 [00:00<06:35, 61.73it/s]

['uh', 'one', 'thought', 'would', 'timid', 'uh', 'turned', 'one', 'stuck', 'first', 'learn']
[array([-0.091699 , -0.039257 ,  1.2013   , -0.52245  ,  0.26734  ,
        0.66546  ,  0.2537   ,  0.31055  , -0.6615   ,  1.3467   ,
       -0.33435  , -0.50338  , -0.62662  , -0.31831  , -0.12121  ,
        0.86012  , -0.89445  ,  0.0071965,  0.48449  , -0.12622  ,
        0.0047007,  0.32844  , -0.58775  ,  0.34265  ,  0.40949  ,
       -0.21937  ,  0.32683  ,  0.25022  , -0.15298  , -1.4369   ,
       -0.024953 , -0.5435   ,  0.14511  , -0.0046536, -0.28092  ,
        0.77723  , -0.36298  , -0.68641  , -0.70413  ,  0.56189  ,
       -0.50438  ,  0.085163 ,  0.31475  ,  1.0738   ,  1.0007   ,
        0.77419  ,  0.16491  ,  0.41478  , -0.69587  , -0.71038  ,
        0.16222  ,  0.56282  ,  0.35479  ,  0.016545 , -0.43079  ,
        0.18744  , -0.69619  , -0.73006  ,  0.34072  ,  0.15146  ,
        0.30226  ,  0.42096  , -0.37714  ,  0.99662  , -1.5978   ,
       -0.67948  , -0.6145   , -0.3

ValueError: operands could not be broadcast together with shapes (300,) (200,) 

In [ ]:
train_set.head()

# Traditional Approach

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression(max_iter=1000)
clf.fit(train_set['combined_embedding'], train_set['label'])

# Evaluate on validation set
preds = clf.predict(dev_set['combined_embedding'])
print(classification_report(dev_set['label'], preds, target_names=['entailment', 'neutral', 'contradiction']))
